<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/04_Apartment_buildings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обнаружение аномалий в начислениях за тепловую энергию

## 4. Многоквартирные дома. Аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами.

### Примеры аномалий

Виды аномалий по показаниям приборов учёта тепловой энергии, которые необходимо выявлять (кроме объектов с видом энергопотребления ГВС (централ):

1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);

2. равные значения показаний в течение нескольких расчетных периодов;

3. снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);

4. аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).

In [1]:
initiate = False
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection/notebooks

/content/anomaly-detection/notebooks


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "..", "src", "anomaly_detection"))

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

from data.make_dataset import load_data
from features.period_anomalies import anomaly_detection_pipeline

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
PATH = "/content/anomaly-detection/"

## 1. Чтение данных

In [6]:
folder_path = '../data/01_raw/'

In [7]:
regenerate = True
data, temperature, buildings = load_data(folder_path, regenerate, path=PATH)

100%|██████████| 24/24 [00:08<00:00,  2.96it/s]


## 2. Оценка аномалий

In [8]:
%%time
period_results = anomaly_detection_pipeline(data, temperature, buildings, path=PATH)

100%|██████████| 24/24 [00:35<00:00,  1.49s/it]

CPU times: user 37.5 s, sys: 656 ms, total: 38.2 s
Wall time: 38.7 s


In [9]:
result = period_results[6]

alpha = 5
beta = 5
threshold = 0.25


q1 = np.percentile(result["Q residuals"], 100 - alpha)
q2 = np.percentile(result["Hotelling's T-squared"], beta)

cond1 = result["Q residuals"] > q1
cond2 = result["Hotelling's T-squared"] < q2
cond3 = result["ниже медианы"]

cond4 = result[f"{threshold}% ниже медианы"]
cond5 = result[f"{threshold}% выше медианы"]

result[((cond1 | cond2) & cond3) & cond4].shape

(30, 16)

In [10]:
result[((cond1 | cond2) & ~cond3) & cond5].shape

(27, 16)